In [22]:
!pip install qiskit
!pip install qiskit_ibm_runtime

In [24]:
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2
from qiskit import QuantumCircuit, transpile
from qiskit.circuit import Parameter
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import PauliEvolutionGate
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt

In [25]:
service = QiskitRuntimeService(channel='ibm_quantum', token=token)#setting the api token from ibm background
backend = service.backend("ibm_sherbrooke")

<ipython-input-25-1427112225>:1: DeprecationWarning: The "ibm_quantum" channel option is deprecated and will be sunset on 1 July. After this date, "ibm_cloud", "ibm_quantum_platform", and "local" will be the only valid channels. Open Plan users should migrate now.  All other users should review the migration guide (https://quantum.cloud.ibm.com/docs/migration-guides/classic-iqp-to-cloud-iqp)to learn when to migrate.
  service = QiskitRuntimeService(channel='ibm_quantum', token=token)


In [26]:
#creating the hamiltonion
H = SparsePauliOp.from_list([("XX", 1.0),("YY", 1.0),("ZZ", 1.0)])  #Qiskit’s way of storing Hamiltonians written as sums of Pauli terms.
t = Parameter("t")
gate = PauliEvolutionGate(H, time=t)

In [27]:
#creating the circuit
qc = QuantumCircuit(2)
qc.x(0)   #intialising
qc.append(gate, [0, 1])
qc.measure_all()

qc_meas = transpile(qc, backend)#rewrites circuit into the specific gate set constraint to the hardware

t_values = np.linspace(0, 5*np.pi, 100)#setting time frame
pub_meas = [(qc_meas, [float(ti)]) for ti in t_values]       #creates a list of circuit executions

In [ ]:
sampler = SamplerV2(backend)#executes and gives  quasi-probability distributions
job = sampler.run(pub_meas, shots=1024)# repeats the circuit 1024 times to collect statistics
result = job.result()#contains all measurement outcomes.



: 

In [ ]:
#calculating probabilty
probability01 = []
probability02 = []

for i in range(len(t_values)):
    meas_array = result[i].data.meas.array.flatten()
    counts = Counter(meas_array)
    total = sum(counts.values())

    p_01 = counts.get(1, 0) / total
    p_10 = counts.get(2, 0) / total
    probability01.append(p_01)#calculating and adding probabilty of 01 sate
    probability02.append(p_10)#calculating and adding probabilty of 01 sate

In [ ]:
#plotting the results
plt.plot(t_values, probability01, label='|01⟩')
plt.plot(t_values, probability02, label='|10⟩')
plt.xlabel("Time")
plt.ylabel("Probability")
plt.title("Time Evolution of 2-Spin Heisenberg Model")
plt.legend()
plt.grid(True)
plt.show()
